In [ ]:
# kaggle_runner.ipynb

import os
import base64
from kaggle_secrets import UserSecretsClient

# 1. Retrieve secrets
user_secrets = UserSecretsClient()
os.environ['GIT_DEPLOY_KEY_B64'] = user_secrets.get_secret("GIT_DEPLOY_KEY_B64")
os.environ['GDRIVE_CREDENTIALS_DATA_B64'] = user_secrets.get_secret("GDRIVE_CREDENTIALS_DATA_B64")

# 2. Run the master shell script
# We make it executable first
!chmod +x /kaggle/working/Alzheimer_classification_mlops/run_pipeline.sh
# Then execute it
!/kaggle/working/Alzheimer_classification_mlops/run_pipeline.sh